# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['M', 'p', 'r', ':', '#']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([12,  4, 40, 26,  2, 11, 23, 13, 36, 19])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[12,  4, 40, 26,  2, 11, 23, 13],
        [ 4, 40, 26,  2, 11, 23, 13, 36],
        [40, 26,  2, 11, 23, 13, 36, 19],
        [26,  2, 11, 23, 13, 36, 19,  2],
        [ 2, 11, 23, 13, 36, 19,  2, 24]])
y training data
tensor([[ 4, 40, 26,  2, 11, 23, 13, 36],
        [40, 26,  2, 11, 23, 13, 36, 19],
        [26,  2, 11, 23, 13, 36, 19,  2],
        [ 2, 11, 23, 13, 36, 19,  2, 24],
        [11, 23, 13, 36, 19,  2, 24, 41]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[ 0.4290,  0.3203,  0.4950,  1.6313,  0.7418, -0.3492,  1.4895,
          -1.2068],
         [ 1.9166,  1.4308,  0.7499,  0.1907, -0.7410, -0.6170, -0.3584,
           0.9260],
         [ 0.6339, -0.1165,  0.1782, -0.4445, -0.9019,  0.1779, -0.2866,
          -0.8229],
         [ 1.5671, -1.3707, -0.2420, -0.3022,  0.9378, -0.4786, -0.5660,
           0.3650]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[ 1.6043, -1.1569,  0.7057,  0.6966,  0.5528, -0.6918, -1.1118,
           1.1643],
         [-1.2173,  1.6943,  1.9158, -0.9976,  0.5491, -0.1318, -0.6315,
           0.1145],
         [-1.0258, -2.0406, -0.1653, -0.0940, -1.2559,  0.7372,  1.6518,
          -0.4367],
         [-0.5087,  0.0979, -1.9084, -2.0764,  0.0269,  0.2530, -1.0435,
          -0.4894],
         [ 0.6339, -0.1165,  0.1782, -0.4445, -0.9019,  0.1779, -0.2866,
          -0.8229],
         [-1.0820,  0.1259, -0.3680,  0.6152, -1.2199, -0.6171, -0.1760,
           0.9911],
         [ 1.1433, -0.2414, -1.1551, -0.7701,  0.1549, -0.4095, -0.7827,
          -1.7803],
         [-0.6405, -0.0025, -0.8935, -0.5778,  1.1398,  0.1831, -0.1390,
          -1.0284]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [18]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [ ]:
d_k = d_q = 4

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension


In [32]:
W_q.weight.shape

torch.Size([4, 8])

In [33]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [34]:
W_q.weight@x_embeddings_total[0][0]

tensor([-0.2515,  0.0128,  0.3557, -0.6198], grad_fn=<MvBackward0>)

In [35]:
W_q(x_embeddings_total)

tensor([[[-0.2515,  0.0128,  0.3557, -0.6198],
         [-0.1177, -0.1718,  0.3718, -0.1644],
         [-0.5848, -0.4304, -0.8143, -0.3974],
         [ 0.4024, -0.4339,  0.3778,  0.2820],
         [ 0.3739,  0.0837,  0.1688,  0.1396],
         [-0.2356, -0.2964, -0.1801,  0.6094],
         [ 1.0784,  0.5525, -0.1745, -0.7631],
         [ 0.7151,  0.5034, -0.1985,  0.0045]],

        [[-0.9608, -0.7332,  1.0046, -0.9668],
         [-0.1344, -0.4106, -0.4176,  0.2494],
         [-0.1996, -0.3655, -0.3098,  0.0319],
         [ 0.7272, -0.0411,  0.4221,  0.1351],
         [-0.5020, -0.4610, -0.0945,  0.0313],
         [ 1.2162,  0.5663,  0.0102,  0.6461],
         [ 0.5693,  0.4314, -0.5739, -1.1411],
         [-0.0806,  0.0345,  0.3882,  0.3679]],

        [[-0.9774, -0.9721,  0.2152, -0.5530],
         [ 0.2509, -0.3457,  0.0869,  0.6787],
         [ 0.1252,  0.0273, -0.2655, -0.1150],
         [-0.1487, -0.5859,  0.1588,  0.0268],
         [ 0.9497,  0.4017,  0.0958,  0.0679],
         

In [36]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [37]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 4]) torch.Size([5, 8, 4]) torch.Size([5, 8, 8])


In [38]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [39]:
print(Q[0][:3])
print(K[0][:3])

tensor([[-0.2515,  0.0128,  0.3557, -0.6198],
        [-0.1177, -0.1718,  0.3718, -0.1644],
        [-0.5848, -0.4304, -0.8143, -0.3974]], grad_fn=<SliceBackward0>)
tensor([[-0.3806,  0.6585, -0.6571,  1.3410],
        [-0.2559, -0.4597, -0.3901, -0.7147],
        [ 0.0371,  0.2333,  1.5068,  0.8984]], grad_fn=<SliceBackward0>)



well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present


ok let's say no linear layer was present, than the Q*K.T would just give me the cosine similarity between static embeddings of tokens

let's say the attention matrix is

`A`

here `A[i][j]` would be the cosine similarity between the ith token and jth token  (here when I say token I mean the embedding of token)

now let's see what would `A@V` do, here I am assuming `V` to is just the static emebdding


`A_softmax` -> is softmax function applied on `A` across each column (why column? cause weights are distributed across column see below)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

here `V[i]` represents the static embedding for the ith word, so `V[i]` is a vector, and ``V`[i]`` is the final vector after attention matrix multiplication

``V`[i]`` we can think of it as change or how the original word gets modified

so when taking the weighted sum it tells us, in direction of which static embedding should the input embedding move to the most and least (depending on the weight), so that it represents that modified vector would represent the meaning of the entire sentence, **now I see the poblem with just using static embedding, the problem is if we just use static embedding to compute cosine similarity it would nudge the input emebedding to this layer in the direction which is most similar to the word which we see in the sentence, which we don't always want**

side note: one more thing I learned is that these query and key vector live in a space which is smaller than the static embedding dimension, so it would help computationaly too

In [40]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])

In [42]:
attention_matrix[0]

tensor([[-0.9607,  0.3626, -0.0272,  1.6227,  0.4415, -0.0248,  0.5533,  0.9070],
        [-0.5330,  0.0815,  0.3681,  0.9711,  0.3078, -0.1483,  0.3450,  0.4630],
        [-0.0587,  0.9493, -1.7062,  1.6502,  0.4142,  0.7721,  0.9899,  0.3414],
        [-0.3090, -0.2524,  0.7362,  0.1065,  0.1546, -0.1218, -0.3984,  0.0544],
        [-0.0109, -0.2998,  0.4132, -0.6840, -0.1598, -0.1150, -0.5866, -0.1439],
        [ 0.8301, -0.1687,  0.1982, -0.5018, -0.1138, -0.1200,  0.3186, -0.6491],
        [-0.9552,  0.0835, -0.7796, -0.6977, -0.2202,  0.5812, -1.6992,  0.5080],
        [ 0.1957, -0.3402, -0.1511, -1.7527, -0.5197,  0.1419, -1.3053, -0.3840]],
       grad_fn=<SelectBackward0>)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

now in self attention when we compute the change required for ith vector we don't want any token above ith token to be deciding how it changes, cause during inference we don't have that information at all so we would need to find a way so that above becomes

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ... + V[i]*A_softmax[i][i]

```

so to do this we convert the lower triangular part equal infinity so that when we do softmax we get zero

In [44]:
print(x_batch[0])
print(y_batch[0])

tensor([12,  4, 40, 26,  2, 11, 23, 13])
tensor([ 4, 40, 26,  2, 11, 23, 13, 36])


In [49]:
torch.ones_like(attention_matrix[0])

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool() # should the diagonal also have no contribution?

tensor([[False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [70]:
attention_matrix[0].masked_fill(torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool(), float('-inf'))

tensor([[-0.9607,  0.3626, -0.0272,  1.6227,  0.4415, -0.0248,  0.5533,  0.9070],
        [   -inf,  0.0815,  0.3681,  0.9711,  0.3078, -0.1483,  0.3450,  0.4630],
        [   -inf,    -inf, -1.7062,  1.6502,  0.4142,  0.7721,  0.9899,  0.3414],
        [   -inf,    -inf,    -inf,  0.1065,  0.1546, -0.1218, -0.3984,  0.0544],
        [   -inf,    -inf,    -inf,    -inf, -0.1598, -0.1150, -0.5866, -0.1439],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -0.1200,  0.3186, -0.6491],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -1.6992,  0.5080],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.3840]],
       grad_fn=<MaskedFillBackward0>)

In [71]:
attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))[0]

tensor([[-0.9607,  0.3626, -0.0272,  1.6227,  0.4415, -0.0248,  0.5533,  0.9070],
        [   -inf,  0.0815,  0.3681,  0.9711,  0.3078, -0.1483,  0.3450,  0.4630],
        [   -inf,    -inf, -1.7062,  1.6502,  0.4142,  0.7721,  0.9899,  0.3414],
        [   -inf,    -inf,    -inf,  0.1065,  0.1546, -0.1218, -0.3984,  0.0544],
        [   -inf,    -inf,    -inf,    -inf, -0.1598, -0.1150, -0.5866, -0.1439],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -0.1200,  0.3186, -0.6491],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -1.6992,  0.5080],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.3840]],
       grad_fn=<SelectBackward0>)

In [72]:
masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

In [73]:
masked_attention_matrix[0]

tensor([[-0.9607,  0.3626, -0.0272,  1.6227,  0.4415, -0.0248,  0.5533,  0.9070],
        [   -inf,  0.0815,  0.3681,  0.9711,  0.3078, -0.1483,  0.3450,  0.4630],
        [   -inf,    -inf, -1.7062,  1.6502,  0.4142,  0.7721,  0.9899,  0.3414],
        [   -inf,    -inf,    -inf,  0.1065,  0.1546, -0.1218, -0.3984,  0.0544],
        [   -inf,    -inf,    -inf,    -inf, -0.1598, -0.1150, -0.5866, -0.1439],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -0.1200,  0.3186, -0.6491],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -1.6992,  0.5080],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.3840]],
       grad_fn=<SelectBackward0>)

In [75]:
import torch.nn.functional as F


F.softmax(masked_attention_matrix[0], dim=-2)

tensor([[1.0000, 0.5698, 0.3744, 0.3612, 0.2412, 0.1464, 0.2016, 0.2414],
        [0.0000, 0.4302, 0.5558, 0.1883, 0.2110, 0.1293, 0.1637, 0.1549],
        [0.0000, 0.0000, 0.0698, 0.3713, 0.2347, 0.3247, 0.3119, 0.1371],
        [0.0000, 0.0000, 0.0000, 0.0793, 0.1810, 0.1328, 0.0778, 0.1029],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1322, 0.1337, 0.0645, 0.0844],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1331, 0.1594, 0.0509],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0212, 0.1620],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0664]],
       grad_fn=<SoftmaxBackward0>)

### should the diagonal (i.e the current token) have any contribution towards update of input embedding vector?

if it does then ``V`[0] = V[0]`` and that would mean the updated embedding would become ``V_updated[0] = 2*V[0]`` need to think more about it